In [1]:
import numpy as np
from numpy.linalg import inv
import csv
import math
import matplotlib.pyplot as plt

In [2]:
#Datasets
mean0, mean1, cov_given = [],[],[]

In [3]:
#Importing Data from Files

with open("./Datasets/DS1_m_0.txt", 'rb') as csv_mean0_file:
    mean0_data = csv.reader(csv_mean0_file, delimiter = ',')
    for row in mean0_data:
        mean0.append(row)
    del(row)

with open("./Datasets/DS1_m_1.txt", 'rb') as csv_mean1_file:
    mean1_data = csv.reader(csv_mean1_file, delimiter = ',')
    for row in mean1_data:
        mean1.append(row)
    del(row)

with open("./Datasets/DS1_Cov.txt", 'rb') as csv_cov_file:
    cov_data = csv.reader(csv_cov_file, delimiter = ',')
    for row in cov_data:
        cov_given.append(row)
    del(row)
    
mean1 = np.delete(mean1, len(mean1[0])-1, 1)
mean1_given = (mean1[0]).astype(np.float)
mean0= np.delete(mean0, len(mean0[0])-1, 1)
mean0_given = (mean0[0]).astype(np.float)
cov_given =  (np.delete(cov_given, len(cov_given[0])-1, 1)).astype(np.float)

In [4]:
#Generating data for class 0
class0 = np.random.multivariate_normal(mean0_given,cov_given, 2000)

#Generating data for class 1
class1 = np.random.multivariate_normal(mean1_given,cov_given, 2000)

In [5]:
#Recalculating Mean and Covariance
mean0_new, mean1_new = [],[]
for i in range(len(class0[0])):
    mean0_new.append(np.mean(class0[:,i]))
    mean1_new.append(np.mean(class1[:,i]))

#Defining mean and Covariance    
mean0 = np.array(mean0_new, dtype = 'float')
mean1 = np.array(mean1_new, dtype = 'float')
cov = np.cov(np.append(class0, class1, axis = 0).T)

In [6]:
#Labelling Class 0
class0 = np.hstack((class0, np.zeros((len(class0) ,1))))

#Labelling Class 1
class1 = np.hstack((class1, np.ones((len(class1),1))))

In [7]:
#Creating training set
X_train = np.append(class0[:int(len(class0)*0.7)], class1[:int(len(class1)*0.7)], axis = 0) 
np.random.shuffle(X_train)
Y_train = X_train[:,-1]
X_train = np.delete(X_train, -1, 1)

#Creating test set
X_test = np.append(class0[int(len(class0)*0.7):], class1[int(len(class1)*0.7):], axis = 0) 
np.random.shuffle(X_test)
Y_test = X_test[:,-1]
X_test = np.delete(X_test, -1, 1)
DS1 = X_test

In [65]:
#Writing Dataset to csv_file
file_name_str = "Dataset_DS1.csv"
myFile = open(file_name_str, 'w') 
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(X_test)

In [12]:
################### Calculating parameters  ########################
prob_C1 = float(np.count_nonzero(Y_train))/float(len(X_train))
prob_C2 = 1 - prob_C1

params = np.dot(inv(cov),(mean0 - mean1))
param0 = -0.5* np.dot(np.dot((mean0.T),  inv(cov)), mean0)  + 0.5* np.dot(np.dot((mean1.T),  inv(cov)), mean1) + math.log(prob_C1/prob_C2)

In [67]:
#Writing params to csv_file
file_name_str = "Params_DS1.csv"
myFile = open(file_name_str, 'w') 
with myFile:
    writer = csv.writer(myFile)
    writer.writerow(params)

In [17]:
#Finding probability of data being in class 0 for training set and adding to list
train_prob_c1_x = []

for row in X_train:
    pred = np.dot(params.T, row) + param0
    train_prob_c1_x.append(1/(1 + math.exp(-pred)))

In [18]:
#Lists used to keep count of data in respective classes
train_count_C1, train_count_C2 = [],[]

for value in train_prob_c1_x:
    if(value >= 0.5):
        train_count_C1.append(value)
    else:
        train_count_C2.append(value)

In [19]:
#Finding probability of data being in class 0 for test set and adding to list
test_prob_c1_x = []

for row in X_test:
    pred = np.dot(params.T, row) + param0
    test_prob_c1_x.append(1/(1 + math.exp(-pred)))

In [20]:
#Lists used to keep count of test data in respective classes
test_count_C1, test_count_C2 = [],[]
pred_class = []
test_TP, test_TN, test_FP, test_FN = 0,0,0,0

for value in test_prob_c1_x:
    if(value >= 0.5):
        test_count_C1.append(value)
        pred_class.append(0)
    else:
        test_count_C2.append(value)
        pred_class.append(1)

In [21]:
#Calculating TP,TN, FP, FN
for i in range(len(pred_class)):
    if(pred_class[i] == Y_test[i]):
        if(pred_class[i] == 0):
            test_TP += 1
        if(pred_class[i] == 1):
            test_TN += 1    
    else:
        if(pred_class[i] == 0):
            test_FP += 1
        if(pred_class[i] == 1):
            test_FN += 1

In [22]:
accuracy = (float(test_TP + test_TN))/(float(len(pred_class)))
precision =  (float(test_TP))/(float(test_TP + test_FP))
recall = (float(test_TP))/(float(test_TP + test_FN))
f_score = (2*precision*recall)/(precision + recall)

In [23]:
print(accuracy)
print(precision)
print(recall)
print(f_score)

0.956666666667
0.953642384106
0.96
0.956810631229


In [58]:
################################## Question3:  K Nearest ###############################################
knn_pred_class = []
value_k = 5

In [59]:
for test in range(len(X_test)):
    min_distance_arr, min_class_arr = [],[]
    for k in range(value_k):
        min_distance = 10000
        min_class = 0
        for train in range(len(X_train)):
            distance = 0
            for i in range(len(X_train[train])):
                distance += (X_train[train][i] - X_test[test][i])**2
            if(len(min_distance_arr) == 0):
                if(distance<min_distance):
                    min_distance = distance
                    min_class = Y_train[train]
            else:
                if(distance<min_distance and distance > min_distance_arr[len(min_distance_arr)-1]):
                    min_distance = distance
                    min_class = Y_train[train]
                    
        if(len(min_distance_arr) < value_k):
            min_distance_arr.append(min_distance)
            min_class_arr.append(min_class)
    
    string = "Iteration: " + str(test) + " Array: " + str(min_distance_arr)
    print(string)
    if(min_class_arr.count(0) > min_class_arr.count(1)):
        knn_pred_class.append(0)
        print(min_class_arr.count(0))
    else:
        knn_pred_class.append(1)
        print(min_class_arr.count(1))

Iteration: 0 Array: [16.996004332242212, 17.24844456719369, 17.988678117534004, 18.064186794949848, 19.335863246983173]
4
Iteration: 1 Array: [13.993981171091834, 16.423358044791943, 16.490395336213574, 16.700590379313965, 17.720095355583801]
4
Iteration: 2 Array: [19.520407125091896, 24.182330576354509, 26.453731208866955, 29.090377974617102, 30.733538273627239]
3
Iteration: 3 Array: [16.281415255042077, 16.303785113436611, 17.279027666457235, 18.10746096846032, 18.549069725906815]
3
Iteration: 4 Array: [15.423863601396699, 18.942892455066954, 20.367648425323281, 21.747871424572637, 22.71137338625519]
3
Iteration: 5 Array: [16.79639717926738, 21.819441528431387, 22.791507218513328, 23.067750910330908, 24.819317494825778]
4
Iteration: 6 Array: [14.451449686200148, 14.625872464445294, 16.01784718763173, 18.115456750730061, 18.922998576731406]
4
Iteration: 7 Array: [32.745133839850226, 36.670947767601405, 41.309917787455696, 41.92232053462368, 43.741124810339429]
4
Iteration: 8 Array: [4

Iteration: 67 Array: [15.714874079838395, 20.21472146077992, 20.886064057923939, 21.961222196377168, 23.928026634049541]
3
Iteration: 68 Array: [11.283215586324626, 14.640805785639941, 14.915478383291203, 15.011794466901975, 15.703942312990458]
3
Iteration: 69 Array: [7.3736871065004106, 16.258118553390883, 17.108121408122365, 17.903514529344871, 18.489310560179984]
3
Iteration: 70 Array: [13.745816372268418, 17.07243076245614, 18.407354577001499, 19.941215650912529, 19.9737365441216]
4
Iteration: 71 Array: [12.564941345927966, 15.218900348191255, 16.171760090654971, 16.790448529281925, 17.068658595724671]
5
Iteration: 72 Array: [14.987009146248935, 19.195263704187635, 19.997190646792848, 23.315005738871186, 23.583887508437499]
3
Iteration: 73 Array: [14.325624440994913, 21.838923008872271, 24.945136672763073, 26.202533623390678, 26.367915158302566]
3
Iteration: 74 Array: [16.731604296762285, 17.260045506569213, 17.291130736883066, 19.290827758736718, 21.495659804161864]
3
Iteration: 7

Iteration: 134 Array: [9.5336494274856296, 11.813410248788816, 12.618675831301651, 14.291740039621029, 14.760139549153424]
3
Iteration: 135 Array: [13.600126088781309, 14.220232740053332, 14.836047533314821, 15.889006015380383, 16.883750464719597]
4
Iteration: 136 Array: [13.199026816342684, 13.278611850208142, 14.482005208055616, 16.902103983755683, 17.102838158165603]
5
Iteration: 137 Array: [17.633647764559282, 17.916626572135801, 20.39473556018191, 20.494236202467004, 20.730867782536897]
3
Iteration: 138 Array: [12.289076405641968, 16.070847884701145, 16.237201462784181, 16.282202899635642, 16.906169229502414]
3
Iteration: 139 Array: [10.024746734463021, 11.437205034214863, 13.630681196820557, 13.730275108984518, 16.25299008085895]
4
Iteration: 140 Array: [11.973249822201012, 13.346827224662473, 13.353118770704814, 14.111461035172081, 14.132527652493172]
3
Iteration: 141 Array: [11.22021178681217, 13.045195262388074, 14.179040169671932, 16.049957301267838, 16.821708851896439]
3
Ite

Iteration: 200 Array: [15.888147013841543, 16.196507143689441, 20.792958646439818, 20.814176572783506, 21.943080444609077]
3
Iteration: 201 Array: [18.345186604446486, 20.185922776560183, 23.66374286692227, 25.5002979512441, 25.787279900765643]
3
Iteration: 202 Array: [12.054704091604465, 14.648089648977432, 17.261218856071803, 17.347219194104255, 18.0776160912054]
3
Iteration: 203 Array: [24.757081908042924, 25.73192705624416, 26.730061708908018, 29.16759314855674, 29.68070152714202]
3
Iteration: 204 Array: [17.04282876212239, 19.653388331848834, 19.672983365773831, 19.844181483923755, 20.728021570606931]
4
Iteration: 205 Array: [10.146843935432491, 10.453761075481516, 12.03475065737239, 12.094036370765629, 12.242921624530773]
4
Iteration: 206 Array: [10.070816845967615, 12.932324388374303, 15.955031710521974, 16.196308545769298, 16.362004563720046]
4
Iteration: 207 Array: [11.926131422265129, 12.329252536203182, 12.434714764948062, 12.663475901825883, 13.420766131525689]
4
Iteration:

Iteration: 266 Array: [13.947417780557403, 16.319158329176094, 20.84875682828741, 21.410869830749405, 21.883500410615543]
3
Iteration: 267 Array: [10.643015080201703, 11.058056727431941, 12.961873312272191, 13.923418999916287, 15.766980892504815]
3
Iteration: 268 Array: [17.748501902742706, 19.363754764376914, 19.893471031645767, 20.314578584116397, 20.498280604309876]
3
Iteration: 269 Array: [23.278543130444671, 24.280116021829709, 25.245236947211829, 28.134703623435605, 28.967182822305176]
3
Iteration: 270 Array: [6.0005139204357629, 12.48185629448102, 12.908418864723833, 14.000027939546998, 14.067762074583104]
3
Iteration: 271 Array: [11.670846920782028, 13.444303975956528, 15.886411545682799, 15.948508001577217, 16.601049370248365]
3
Iteration: 272 Array: [15.122098650171871, 15.320867919256862, 16.05618035059258, 17.344680989155641, 18.577961702011116]
4
Iteration: 273 Array: [9.6185184426170469, 11.06194060716709, 11.785513695426934, 12.323537219789879, 12.866564925204347]
3
Iter

...

Iteration: 926 Array: [8.5875861554518398, 16.126542396166005, 22.919545344996077, 23.675166653997035, 24.231049894227883]
3
Iteration: 927 Array: [13.838632917074547, 14.109180572295074, 17.337262084712791, 18.191726707724182, 18.279054666943839]
4
Iteration: 928 Array: [16.80121564411813, 19.937010022004959, 20.556888861308614, 21.766352504196401, 22.757751351798486]
3
Iteration: 929 Array: [8.9416794551916485, 12.754413476199813, 14.987404257218779, 15.141008558655709, 15.517549983586001]
3
Iteration: 930 Array: [16.597376577743308, 18.253619640179679, 21.160427390698569, 21.743343994708358, 22.100904465371105]
4
Iteration: 931 Array: [13.254374316975044, 17.828459606289293, 17.991921107176374, 18.066578770219337, 18.093635418894745]
4
Iteration: 932 Array: [15.13960446279911, 15.785551186926538, 16.493211362257565, 16.537068994302146, 17.079943811293163]
3
Iteration: 933 Array: [17.212400539147616, 17.899325750440973, 18.837519610459516, 18.975843238684384, 19.24657403580602]
5
Ite

Iteration: 992 Array: [12.863004788352296, 22.530394021640898, 23.879680718526487, 25.66672149532285, 25.891893876814073]
4
Iteration: 993 Array: [40.082011753850828, 50.94984224315106, 51.253047234832408, 51.325597037339108, 51.570099455193422]
4
Iteration: 994 Array: [10.649409440232352, 13.374031409366296, 14.697529217022874, 14.93265440622271, 15.240771105962336]
3
Iteration: 995 Array: [17.145519308060322, 18.971501713587926, 20.033974460846018, 20.533595489860417, 22.288968949240925]
3
Iteration: 996 Array: [17.994250986493682, 21.218589206436423, 22.788863158513422, 23.54328684999734, 23.830103776024206]
5
Iteration: 997 Array: [8.4757029021050876, 9.4475951385924226, 11.46915500842913, 12.735634055706116, 12.809230892124374]
3
Iteration: 998 Array: [17.061210338006617, 17.200409989935448, 19.319730304519872, 20.20770406119529, 21.539225448994262]
4
Iteration: 999 Array: [19.493182890474067, 21.496821421521819, 24.010535223349127, 24.940819409177312, 25.963860175670543]
3
Iterat

Iteration: 1058 Array: [13.718315974093157, 14.101896087041034, 14.164644254473199, 14.844538932938441, 17.026965535596709]
4
Iteration: 1059 Array: [26.381646666255016, 30.097870811509132, 32.405591171214766, 33.688505776081115, 35.430603188641854]
4
Iteration: 1060 Array: [10.808340981104765, 13.535808734554985, 15.740886681614352, 16.959763514959793, 17.847135448638515]
3
Iteration: 1061 Array: [14.989570445653365, 23.073283547330742, 23.982291056891807, 25.989368714803263, 27.097130779511659]
4
Iteration: 1062 Array: [11.304980602061718, 15.874201271741315, 16.186020262082721, 18.328663560250988, 18.76307711163788]
3
Iteration: 1063 Array: [26.206431449366093, 26.441421732326305, 26.751297155240408, 28.964931514501462, 30.474311808681009]
5
Iteration: 1064 Array: [10.784428801620329, 11.607595264488083, 12.087659760365394, 12.517513926073466, 13.682983298898284]
3
Iteration: 1065 Array: [15.231358628537182, 18.04150784131803, 19.618945263404779, 21.077829973316717, 21.3938809075814

Iteration: 1124 Array: [30.034254217304635, 32.695914402678838, 39.483398142108697, 39.737914011896159, 41.036166226754446]
5
Iteration: 1125 Array: [19.250681316355823, 20.866467704241806, 21.518984388116436, 24.572011279665155, 26.274071366606421]
5
Iteration: 1126 Array: [18.942159259272255, 19.462834186975403, 21.13500487380459, 22.83407175894752, 24.091949372677902]
5
Iteration: 1127 Array: [26.099663430526931, 27.704899563304906, 29.535700236450079, 31.809709208259328, 34.009237077608098]
3
Iteration: 1128 Array: [4.4983850539722372, 8.7924109797529688, 11.419680937493542, 12.137028325772471, 12.57912063167691]
3
Iteration: 1129 Array: [10.498010258758754, 11.285813990960424, 11.321421594464091, 11.778600634609761, 12.109646262841116]
5
Iteration: 1130 Array: [16.883320499152422, 17.621835498284479, 20.051828044524488, 20.461924189065591, 20.738152125407275]
3
Iteration: 1131 Array: [7.4467250803258311, 10.803885161871232, 12.483580485124923, 13.754053268682881, 14.01779201123085

Iteration: 1190 Array: [13.050417077086278, 13.430029187419837, 14.278867884523388, 15.041721387560782, 15.277970101439349]
3
Iteration: 1191 Array: [10.327076145436966, 13.006401404516971, 16.157258723057321, 16.289275133896897, 16.883930280681824]
3
Iteration: 1192 Array: [12.650091882068644, 12.812602054862756, 15.390599538545965, 15.743845751059942, 16.085039693388573]
3
Iteration: 1193 Array: [20.833906025238967, 21.850401091072694, 22.729881049243119, 24.371386739230271, 24.893358773364294]
5
Iteration: 1194 Array: [13.190764860568247, 20.136266651569997, 22.423426966617633, 23.595572643475322, 26.196784662710314]
4
Iteration: 1195 Array: [8.4823595921388648, 9.1990771908677402, 11.029190146249922, 12.055333121030587, 14.660165767300651]
3
Iteration: 1196 Array: [9.2010833743648401, 23.410681653737875, 24.720123032242373, 24.777082106412564, 25.540972376744175]
3
Iteration: 1197 Array: [18.012261551030317, 21.612863030693923, 23.135372516746564, 25.270311671232527, 27.12054194989

In [53]:
len(knn_pred_class)

1200

In [60]:
knn_test_TP, knn_test_TN, knn_test_FP, knn_test_FN = 0,0,0,0

#Calculating TP,TN, FP, FN
for i in range(len(knn_pred_class)):
    if(knn_pred_class[i] == Y_test[i]):
        if(knn_pred_class[i] == 0):
            knn_test_TP += 1
        if(knn_pred_class[i] == 1):
            knn_test_TN += 1    
    else:
        if(knn_pred_class[i] == 0):
            knn_test_FP += 1
        if(knn_pred_class[i] == 1):
            knn_test_FN += 1

In [61]:
knn_accuracy = (float(knn_test_TP + knn_test_TN))/(float(len(knn_pred_class)))
knn_precision =  (float(knn_test_TP))/(float(knn_test_TP + knn_test_FP))
knn_recall = (float(knn_test_TP))/(float(knn_test_TP + knn_test_FN))
knn_f_score = (2*knn_precision*knn_recall)/(knn_precision + knn_recall)

In [62]:
print(knn_accuracy)
print(knn_precision)
print(knn_recall)
print(knn_f_score)

0.545
0.545302013423
0.541666666667
0.54347826087
